# Capstone Project

## Modeling Notebook - `PREPOST_dataset_COMBO` from 11/14/19

#### Importing Libraries

In [1]:
%matplotlib inline

# general libraries
import re
import string
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing date libraries
import datetime as dt
import dateutil.parser as dparser

# scikit-learn libraries for preprocessing
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

# scikit-learn libraries for constructing pipelines
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import clone

# scikit-learn libraries for clustering and dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.mixture import GaussianMixture

# scikit-learn libraries for evaluation
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# scikit-learn libraries for feature selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import RFECV

# scikit-learn libraries for learning
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_validate, cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# saving models
import pickle
from sklearn.externals import joblib

# setting pandas display options
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 10000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('precision', 5)
pd.options.mode.chained_assignment = None

/anaconda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


#### Directory/File Structure

In [2]:
sys.version

'3.6.9 |Anaconda custom (x86_64)| (default, Jul 30 2019, 13:42:17) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
print ('Running pandas version:', pd.__version__)
print ('Running numpy version:', np.__version__)
print ('Running sklearn version:', sklearn.__version__)

Running pandas version: 0.25.2
Running numpy version: 1.17.2
Running sklearn version: 0.21.3


In [4]:
os.getcwd()

'/Users/Shailesh/Desktop/MIDS/Capstone/w210CapstoneProject/notebooks'

In [5]:
os.chdir('../data')

In [6]:
sorted(os.listdir())

['.DS_Store',
 'Capstone_Fall_Shannon_Sept2019_request.csv',
 'POSTOP_ALL_col_names_11_9.pkl',
 'POSTOP_TREE_ALL_col_names_11_9.pkl',
 'POSTOP_categorical_TREE_col_names_11_14.pkl',
 'POSTOP_categorical_TREE_col_names_11_9.pkl',
 'POSTOP_categorical_TREE_col_names_12_1.pkl',
 'POSTOP_categorical_col_names_11_14.pkl',
 'POSTOP_categorical_col_names_11_9.pkl',
 'POSTOP_categorical_col_names_12_1.pkl',
 'POSTOP_numerical_col_names_11_14.pkl',
 'POSTOP_numerical_col_names_11_9.pkl',
 'POSTOP_numerical_col_names_12_1.pkl',
 'PREOP_ALL_col_names_11_9.pkl',
 'PREOP_TREE_ALL_col_names_11_9.pkl',
 'PREOP_categorical_TREE_col_names_11_14.pkl',
 'PREOP_categorical_TREE_col_names_11_9.pkl',
 'PREOP_categorical_TREE_col_names_12_1.pkl',
 'PREOP_categorical_col_names_11_14.pkl',
 'PREOP_categorical_col_names_11_9.pkl',
 'PREOP_categorical_col_names_12_1.pkl',
 'PREOP_dataset_10_24.pkl',
 'PREOP_dataset_TREE_10_24.pkl',
 'PREOP_dataset_TREE_cabg.pkl',
 'PREOP_dataset_TREE_cabvalve.pkl',
 'PREOP_datas

## Preprocessing Unprocessed Dataset

### Loading Dataset

#### `PREPOST_dataset_COMBO_11_14`

In [7]:
data = pd.read_pickle('../data/PREPOST_dataset_COMBO_12_1.pkl')

In [8]:
data.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,recordId,predstro,cnstrokp,cnstrokttia,cncomaenceph,strokeBin,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,54,180.0,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,7,4,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,1.0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,3,3,0,4,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0.014,2,0,2,0,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,28.0,29.0,150.0,1.2,NaN,NaN,108.0,1.0,0.0,0.0,0.0,0.0,

In [9]:
data.shape

(42740, 254)

In [10]:
columns = data.columns

In [11]:
data= pd.read_csv('../dataWithProcedureType.csv')

In [12]:
data.shape

(42740, 263)

In [13]:
newColumns = data.columns
for col in newColumns:
    if col not in columns and col != 'procTyp':
        print(col)
        data.drop(col, axis=1, inplace=True)

mtopd
mtcause_Cardiac
mtcause_Infection
mtcause_Neuro
mtcause_Pulmonary
mtcause_Renal
mtcause_Vascular
mtcause


In [14]:
data.shape

(42740, 255)

In [15]:
data.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,recordId,predstro,cnstrokp,cnstrokttia,cncomaenceph,strokeBin,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn,procTyp
0,54,180.0,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,7,4,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,3,3,0,4,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0.014,2,0,2,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,28.0,29.0,150.0,1.2,NaN,NaN,108.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,

In [16]:
cabg = data.loc[data['procTyp'] == 1]
valve = data.loc[data['procTyp'] == 2]
cabgValve = data.loc[data['procTyp'] == 3]

In [17]:
cabg.shape

(28945, 255)

In [18]:
valve.shape

(8500, 255)

In [19]:
cabgValve.shape

(5295, 255)

In [20]:
origData = data

In [21]:
data = valve

### Dropping Irrelevant Columns

- `recordId`
- `cnstrokp`
- `cnstrokttia`
- `cncomaenceph`
- `strokeBin` as we are going to use the more inclusive `strokeBin2` as our outcome variable

#### Relative to prior versions, we are also going to retain `predstro` that will be used to compare our models versus the `STS` Model

In [22]:
cols_to_drop = ['recordId',
                'cnstrokp',
                'cnstrokttia',
                'cncomaenceph',
                'strokeBin',
               'procTyp']

In [23]:
len(cols_to_drop)

6

- dropping columns

In [24]:
data = data.drop(cols_to_drop, axis=1)

In [25]:
data.shape

(8500, 249)

- resetting `DataFrame` `index` 

In [26]:
data = data.reset_index(drop=True)

In [27]:
data.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,predstro,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,55,175.30000,125.9,40.96962,47.0,2.6,3.7,6.4,16.5,50.0,70.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,7,0,2,1,1,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,3,2,2,0,1,0,0,0,0,0,0,0,0,0,0,0.014,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,34.0,31.2,148.0,2.6,NaN,NaN,119.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,

In [28]:
data.tail()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,predstro,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
8495,72,165.10001,64.0,23.47933,36.5,0.77,4.1,5.6,6.40,60.0,NaN,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,11,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.013,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,25.0,33.3,115.0,0.7,NaN,NaN,89.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [29]:
data.shape

(8500, 249)

### Feature Matrix `X`
- in the next cell you can select which outcome vector you want to use: `strokeBin` or `strokeBin2`

In [30]:
X = data.copy().drop('strokeBin2', axis=1)

In [31]:
X = X.reset_index(drop=True)

In [32]:
X.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,predstro,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,55,175.3,125.9,40.96962,47.0,2.6,3.7,6.4,16.5,50.0,70.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,7,0,2,1,1,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,3,2,2,0,1,0,0,0,0,0,0,0,0,0,0,0.014,NaN,0,0,NaN,NaN,NaN,NaN,NaN,34.0,31.2,148.0,2.6,NaN,NaN,119.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0

In [33]:
X.shape

(8500, 248)

### Target Vector `y`

In [34]:
y = data.copy()['strokeBin2']

In [35]:
type(y)

pandas.core.series.Series

In [36]:
y.head(2)

0    0
1    0
Name: strokeBin2, dtype: int64

In [37]:
y.shape

(8500,)

In [38]:
y.unique()

array([0, 1])

In [39]:
y.value_counts()

0    8353
1     147
Name: strokeBin2, dtype: int64

### `X_train_all`, `X_devtest`, `y_train`, `y_devtest`
- using `train_test_split` with `stratify` parameter to ensure relative proportion of outcome classes are the same in `train`, `dev` and `test` sets
- observation split will be `80/10/10` between `train`, `dev`, `test`
- designation of `_all` denotes complete feature set (`PRE` + `POST`)

In [40]:
X_train_all, X_test_all, y_train, y_test = train_test_split(X,
                                                                  y,
                                                                  test_size=0.1,
                                                                  random_state=0,
                                                                  stratify=y)

#### validating `train_test_split`

In [41]:
X.shape

(8500, 248)

In [42]:
np.rint(X.shape[0] * 0.20)

1700.0

In [43]:
X_train_all.shape, X_test_all.shape, y_train.shape, y_test.shape

((7650, 248), (850, 248), (7650,), (850,))

In [44]:
X.shape[0] - X_train_all.shape[0] - X_test_all.shape[0]

0

In [45]:
y.shape[0] - y_train.shape[0] - y_test.shape[0]

0

#### validating `stratify` worked
- relative proportion of classes in `y`

In [46]:
print (np.round(y.value_counts()[0] / y.shape[0], 4))
print (np.round(y.value_counts()[1] / y.shape[0], 4))

0.9827
0.0173


- relative proportion of classes in `y_train`

In [47]:
print (np.round(y_train.value_counts()[0] / y_train.shape[0], 4))
print (np.round(y_train.value_counts()[1] / y_train.shape[0], 4))

0.9827
0.0173


- relative proportion of classes in `y_devtest`

In [48]:
print (np.round(y_test.value_counts()[0] / y_test.shape[0], 4))
print (np.round(y_test.value_counts()[1] / y_test.shape[0], 4))

0.9824
0.0176


### `X_dev_all`, `X_test_all`, `y_dev`, `y_test`

In [49]:
# X_dev_all, X_test_all, y_dev, y_test = train_test_split(X_devtest_all,
#                                                         y_devtest,
#                                                         test_size=0.5,
#                                                         random_state=0,
#                                                         stratify=y_devtest)

- validating `train_test_split`

In [50]:
# X_devtest_all.shape

In [51]:
# np.rint(X_devtest_all.shape[0] * 0.50)

In [52]:
# X_dev_all.shape, X_test_all.shape, y_dev.shape, y_dev.shape

#### validating `stratify` worked
- relative proportion of classes in `y`

In [53]:
# print (np.round(y.value_counts()[0] / y.shape[0], 4))
# print (np.round(y.value_counts()[1] / y.shape[0], 4))

- relative proportion of classes in `y_devtest`

In [54]:
# print (np.round(y_devtest.value_counts()[0] / y_devtest.shape[0], 4))
# print (np.round(y_devtest.value_counts()[1] / y_devtest.shape[0], 4))

- relative proportion of classes in `y_dev`

In [55]:
# print (np.round(y_dev.value_counts()[0] / y_dev.shape[0], 4))
# print (np.round(y_dev.value_counts()[1] / y_dev.shape[0], 4))

- relative proportion of classes in `y_test`

In [56]:
# print (np.round(y_test.value_counts()[0] / y_test.shape[0], 4))
# print (np.round(y_test.value_counts()[1] / y_test.shape[0], 4))

### Resetting Indicies

In [57]:
X_train_all = X_train_all.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [58]:
# X_dev_all = X_dev_all.reset_index(drop=True)
# y_dev = y_dev.reset_index(drop=True)

In [59]:
X_test_all = X_test_all.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

### Extracting `predstro` from `X_train`, `X_dev` and `X_test`
- and subsequently creating `predstro_train`, `predstro_dev`, `predstro_test`

- `train`

In [60]:
X_train_all.shape

(7650, 248)

In [61]:
predstro_train = X_train_all['predstro']

In [62]:
predstro_train.shape

(7650,)

In [63]:
X_train_all = X_train_all.drop('predstro', axis=1)

In [64]:
X_train_all.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,59,177.00000,71.0,22.66271,45.0,0.9,4.0,5.8,7.5,60.0,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,3,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,33.0,28.0,179.0,0.9,NaN,NaN,111.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0

In [65]:
X_train_all.shape

(7650, 247)

- `dev`

In [66]:
#X_dev_all.shape

In [67]:
#predstro_dev = X_dev_all['predstro']

In [68]:
#predstro_dev.shape

In [69]:
#X_dev_all = X_dev_all.drop('predstro', axis=1)

In [70]:
#X_dev_all.head(2)

In [71]:
#X_dev_all.shape

- `test`

In [72]:
X_test_all.shape

(850, 248)

In [73]:
predstro_test = X_test_all['predstro']

In [74]:
predstro_test.shape

(850,)

In [75]:
X_test_all = X_test_all.drop('predstro', axis=1)

In [76]:
X_test_all.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,73,174.0,86.0,28.40534,43.9,1.01,4.3,6.0,8.54,63.0,35.0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,12,3,3,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,30.4,34.9,81.0,1.1,NaN,NaN,70.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,2,0,0

In [77]:
X_test_all.shape

(850, 247)

- validating row count - `42,740` total observations

In [78]:
42740 - X_train_all.shape[0]  - X_test_all.shape[0]#- X_dev_all.shape[0]

34240

#### Last look at the data (`X_train_all`) before modeling

In [79]:
X_train_all.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,59,177.00000,71.0,22.66271,45.0,0.9,4.0,5.8,7.50,60.0,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,3,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,33.0,28.0,179.0,0.9,NaN,NaN,111.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,

In [80]:
X_train_all.tail()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
7645,75,185.39999,122.3,35.58009,46.0,1.10,4.4,5.90,13.30,58.0,33.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,6,1,2,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,0.0,4.0,2.0,0.0,26.8,32.6,145.0,1.1,NaN,NaN,102.0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,

## Pre-processing Data using `Scikit-Learn` `Pipelines`

### Creating a `FeatureSelector` transformer
- necessary because we are working with heterogeneous data (numerical and categorical features) -- want to be able to pick and choose which features (columns) to pass through our pipelines (and transform them) instead of having to pass through the whole dataframe

In [81]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    '''
    Transformer to select column from a data frame to perform additional transformations on
    Use this for selecting column(s) that require fit transform
    '''
    
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[data_dict.columns.intersection(self.key)]

## Loading Column Names for `PREOP` and `POSTOP` Features
- these `lists` of features will be used to mask/select specified features from the `COMBO` dataset to assemble `PREOP`, `PREOP_TREE`, `POSTOP` and `POSTOP_TREE` feature sets as appropriate
- these are essentially helper lists as `DataFrames` put through `Pipelines` emerge as `numpy` arrays which do not have column names
- can convert resulting `numpy` arrays back into `DataFrames` for further analysis

#### `PREOP_dataset` Column Names

- `DUMMIES` Version = `PREOP_numerical_col_names` + `PREOP_categorical_col_names`
- `TREE` Version    = `PREOP_numerical_col_names` + `PREOP_categorical_TREE_col_names`

- `PREOP_numerical_col_names`

In [82]:
with open('PREOP_numerical_col_names_12_1.pkl', 'rb') as filehandle:
    PREOP_numerical_col_names = pickle.load(filehandle)

In [83]:
PREOP_numerical_col_names[0:5]

['age', 'heightcm', 'weightkg', 'bmi', 'hct']

In [84]:
len(PREOP_numerical_col_names)

11

- `PREOP_categorical_col_names`

In [85]:
with open('PREOP_categorical_col_names_12_1.pkl', 'rb') as filehandle:
    PREOP_categorical_col_names = pickle.load(filehandle)

In [86]:
PREOP_categorical_col_names[0:5]

['surgdt_month_Jan',
 'surgdt_month_Feb',
 'surgdt_month_Mar',
 'surgdt_month_Apr',
 'surgdt_month_May']

In [87]:
len(PREOP_categorical_col_names)

99

- `PREOP_categorical_TREE_col_names`

In [88]:
with open('PREOP_categorical_TREE_col_names_12_1.pkl', 'rb') as filehandle:
    PREOP_categorical_TREE_col_names = pickle.load(filehandle)

In [89]:
PREOP_categorical_TREE_col_names[0:5]

['surgdt_month',
 'surgdt_DayOfWeek',
 'surgdt_PartOfMonth',
 'gender',
 'racecaucasian']

In [90]:
len(PREOP_categorical_TREE_col_names)

61

#### `POSTOP_dataset` Column Names

- `DUMMIES` Version = `POSTOP_numerical_col_names` + `POSTOP_categorical_col_names`
- `TREE` Version = `POSTOP_numerical_col_names` + `POSTOP_categorical_TREE_col_names`

In [91]:
with open('POSTOP_numerical_col_names_12_1.pkl', 'rb') as filehandle:
    POSTOP_numerical_col_names = pickle.load(filehandle)

In [92]:
with open('POSTOP_categorical_col_names_12_1.pkl', 'rb') as filehandle:
    POSTOP_categorical_col_names = pickle.load(filehandle)

In [93]:
with open('POSTOP_categorical_TREE_col_names_12_1.pkl', 'rb') as filehandle:
    POSTOP_categorical_TREE_col_names = pickle.load(filehandle)

## Functions Used in Pipelines

#### `SimpleImputer(missing_values=np.nan, strategy='median'` 
- to replace numerical feature `NaN`s with `X_train` numerical feature `NaN`s

#### `PolynomialFeatures()`
- per Albon 13.2 pages 225-227, can use `PolynomialFeatures()` to create interaction terms
- Parameter `interaction_only=True` tells `PolynomialFeatures()` to ONLY return interaction terms (and not polynomial features
- By default, `PolynomialFeatures()` will add a feature containing ones (a vector of `1`s) called a `bias` -- we can prevent that through the parameter `include_bias=False`
- The `degree` parameter determines the maximum number of features to create interaction terms from (in case we wanted to create an interaction term of `n_features`)
- open question whether we should apply `PolynomialFeatures` to the entire feature matrix or only to the numerical and/or categorical features and how we should do it (with everything higher degree + interaction terms or just interaction terms).
- Given the number of featues we have, we will have many more features after transforming with `PolynomialFeatures`
- The formula for calculating the number of the polynomial features is `N(n,d)=C(n+d,d)` where `n` is the number of the features, `d` is the degree of the polynomial, `C` is binomial coefficient(combination). In our case the number is `C(3+2,2)=5!/(5-2)!2!=10` but when the number of features or the degree is high the polynomial features becomes too many.

#### `sklearn.preprocessing` scalers such as `StandardScaler()`

#### Custom `helper` functions, such as `convert_df_to_numpy`
- need to create a function to convert `DataFrame` to `numpy.ndarray` then can use `FeatureUnion` to combine with `numpy.ndarray` that results from `numerical_pipeline` to form feature matrix

In [94]:
def convert_df_to_numpy(df):
    
    df = df.values
    
    return df

#### `FeatureUnion`
- to reassemble feature matrix from the product of multiple `pipelines`

# `PREOP` Feature Matrix Pipelines

### `PREOP_numerical_features` Pipeline

In [95]:
PREOP_numerical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_numerical_col_names)), 
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler()) 
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [96]:
PREOP_numerical = PREOP_numerical_features_pipeline.fit_transform(X_train_all)

In [97]:
PREOP_numerical.shape, len(PREOP_numerical_col_names)

((7650, 11), 11)

### `PREOP_categorical_features_pipeline`

In [98]:
PREOP_categorical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_categorical_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [99]:
PREOP_categorical = PREOP_categorical_features_pipeline.fit_transform(X_train_all)

In [100]:
PREOP_categorical.shape, len(PREOP_categorical_col_names)

((7650, 99), 99)

## Assembling `PREOP_feature_matrix` via `FeatureUnion`

In [101]:
PREOP_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                     ('PREOP_cat_features', PREOP_categorical_features_pipeline)
                                    ])

In [102]:
type(PREOP_feature_matrix)

sklearn.pipeline.FeatureUnion

### Transforming `X_train_all` to get `X_train_PREOP`

In [103]:
X_train_PREOP = PREOP_feature_matrix.fit_transform(X_train_all)

- validating

In [104]:
X_train_PREOP.shape, y_train.shape

((7650, 110), (7650,))

- putting into a `DataFrame` for easy analysis

In [105]:
PREOP_feature_col_names = PREOP_numerical_col_names + PREOP_categorical_col_names

In [106]:
len(PREOP_feature_col_names), len(PREOP_numerical_col_names), len(PREOP_categorical_col_names)

(110, 11, 99)

In [107]:
X_train_PREOP = pd.DataFrame(X_train_PREOP,
                             columns=PREOP_feature_col_names)

In [108]:
X_train_PREOP.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%
0,-0.51204,0.60592,-0.75915,-0.74853,1.12084,-0.23094,0.15716,-0.08171,-0.29361,0.28612,-0.17594,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.51953,1.13210,0.26894,-0.24250,0.57404,-0.12853,-0.78372,-0.38966,-0.01054,0.01630,-0.17594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.66704,-1.17773,0.07267,0.56756,-0.79297,-0.33336,-0.21919,0.32889,-0.53578,-0.16359,-0.17594,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.82204,0.24919,-0.15164,-0.21633,-0.95701,0.28114,-0.03101,0.12359,0.92986,-3.08665,1.93264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.18545,1.76530,0.31567,-0.41603,-0.28262,0.17872,0.53352,-0.08171,1.19720,-2.23221,0.47286,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Transforming `X_dev_all` to get `X_dev_PREOP`

In [109]:
#X_dev_PREOP = PREOP_feature_matrix.transform(X_dev_all)

- validating

In [110]:
#X_dev_PREOP.shape, y_dev.shape

- putting into a `DataFrame` for easy analysis

In [111]:
# X_dev_PREOP = pd.DataFrame(X_dev_PREOP,
#                            columns=PREOP_feature_col_names)

In [112]:
# X_dev_PREOP.head()

### Transforming `X_test_all` to get `X_test_PREOP`

In [113]:
X_test_PREOP = PREOP_feature_matrix.transform(X_test_all)

- validating

In [114]:
X_test_PREOP.shape, y_test.shape

((850, 110), (850,))

- putting into a `DataFrame` for easy analysis

In [115]:
X_test_PREOP = pd.DataFrame(X_test_PREOP,
                            columns=PREOP_feature_col_names)

In [116]:
X_test_PREOP.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%
0,0.57294,0.33838,-0.05818,-0.18295,0.92035,-0.11828,0.72170,0.12359,0.03349,0.55594,-0.33813,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.26295,0.67727,2.30176,1.27202,0.75631,0.07631,-0.21919,0.94479,1.40478,-0.16359,3.14912,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.10795,-1.81093,-0.46941,0.40320,-1.06637,-0.43577,-0.03101,1.04744,-0.63957,-1.24287,-0.17594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.88294,0.90023,0.32035,-0.12964,-0.24616,-0.26167,-0.59555,-0.80027,-0.18982,-1.06299,-0.17594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.72794,0.42756,0.08202,-0.11836,1.30311,-0.23094,0.15716,-0.18436,-0.30304,-0.16359,-0.74363,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### NOTE:
- did not have to put feature matricies into `DataFrames` - could have used directly in pipelines, but wanted to put into `DataFrames` in case wanted to retrieve feature names

# `POSTOP` Feature Matrix Pipelines

### `POSTOP_numerical_features` Pipeline

In [117]:
POSTOP_numerical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_numerical_col_names)), 
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler()) 
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [118]:
POSTOP_numerical = POSTOP_numerical_features_pipeline.fit_transform(X_train_all)

In [119]:
POSTOP_numerical.shape, len(POSTOP_numerical_col_names)

((7650, 15), 15)

### `POSTOP_categorical_features_pipeline`

In [120]:
POSTOP_categorical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_categorical_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [121]:
POSTOP_categorical = POSTOP_categorical_features_pipeline.fit_transform(X_train_all)

In [122]:
POSTOP_categorical.shape, len(POSTOP_categorical_col_names)

((7650, 96), 96)

## Assembling `POSTOP_feature_matrix` via `FeatureUnion`

In [123]:
POSTOP_feature_matrix = FeatureUnion([('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                      ('POSTOP_cat_features', POSTOP_categorical_features_pipeline)
                                     ])

In [124]:
type(POSTOP_feature_matrix)

sklearn.pipeline.FeatureUnion

### Transforming `X_train_all` to get `X_train_POSTOP`

In [125]:
X_train_POSTOP = POSTOP_feature_matrix.fit_transform(X_train_all)

- validating

In [126]:
X_train_POSTOP.shape, y_train.shape

((7650, 111), (7650,))

- putting into a `DataFrame` for easy analysis

In [127]:
POSTOP_feature_col_names = POSTOP_numerical_col_names + POSTOP_categorical_col_names

In [128]:
len(POSTOP_feature_col_names), len(POSTOP_numerical_col_names), len(POSTOP_categorical_col_names)

(111, 15, 96)

In [129]:
X_train_POSTOP = pd.DataFrame(X_train_POSTOP,
                              columns=POSTOP_feature_col_names)

In [130]:
X_train_POSTOP.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,1.65960,-2.48173,1.15944,-0.38721,0.0375,0.03141,0.61150,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,-0.09801,-0.09217,-0.00525,-0.10154,6.73500,2.18333,-0.95864,-0.03123,-0.51176,3.86140,-0.30441,0.0375,0.03141,0.92041,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,-0.66530,0.02998,0.52833,-0.55283,0.0375,0.03141,0.47109,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,-0.49621,-0.56101,0.46916,0.52367,0.0375,0.03141,0.13410,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,-0.09801,-0.09217,-0.00525,-0.10154,2.09268,-0.26517,-1.96242,-0.03123,0.47322,0.86361,0.02682,0.0375,0.03141,1.45398,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0

### Transforming `X_dev_all` to get `X_dev_POSTOP`

In [131]:
# X_dev_POSTOP = POSTOP_feature_matrix.transform(X_dev_all)

- validating

In [132]:
# X_dev_POSTOP.shape, y_dev.shape

- putting into a `DataFrame` for easy analysis

In [133]:
# X_dev_POSTOP = pd.DataFrame(X_dev_POSTOP,
#                             columns=POSTOP_feature_col_names)

In [134]:
# X_dev_POSTOP.head()

### Transforming `X_test_all` to get `X_test_POSTOP`

In [135]:
X_test_POSTOP = POSTOP_feature_matrix.transform(X_test_all)

- validating

In [136]:
X_test_POSTOP.shape, y_test.shape

((850, 111), (850,))

- putting into a `DataFrame` for easy analysis

In [137]:
X_test_POSTOP = pd.DataFrame(X_test_POSTOP,
                             columns=POSTOP_feature_col_names)

In [138]:
X_test_POSTOP.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,1.11008,0.91647,-0.77335,-0.22160,0.03750,0.03141,-0.53988,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,0.60283,-1.25050,1.67222,0.27524,0.03750,0.03141,1.17315,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,-0.09801,-0.01302,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,-0.24259,0.91647,-0.41835,-0.47002,-1.32988,0.30571,-0.20289,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,0.72964,0.32548,-0.73390,-0.30441,0.03750,0.03141,-0.42755,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,0.81419,-0.51176,-0.26057,-0.47002,0.03750,0.03141,-0.90495,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0

## Assembling `PREOP + POSTOP` Combined Feature Matrix

#### -  Using `FeatureUnion | Pipeline` to Create Combined `PREOP + POSTOP` Feature Matrix

In [139]:
PREPOST_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                       ('PREOP_cat_features', PREOP_categorical_features_pipeline),
                                       ('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                       ('POSTOP_cat_features', POSTOP_categorical_features_pipeline)
                                      ])

- can use the code pattern above to create `X_train_PREPOST`, `X_dev_PREPOST`, `X_test_PREPOST`

In [140]:
PREPOST_feature_col_names = PREOP_feature_col_names + POSTOP_feature_col_names

In [141]:
len(PREPOST_feature_col_names), len(PREOP_feature_col_names), len(POSTOP_feature_col_names)

(221, 110, 111)

### Transforming `X_train_all` to get `X_train_PREPOST`

In [142]:
X_train_PREPOST = PREPOST_feature_matrix.fit_transform(X_train_all)

- validating

In [143]:
X_train_PREPOST.shape, y_train.shape

((7650, 221), (7650,))

- putting into a `DataFrame` for easy analysis

In [144]:
X_train_PREPOST = pd.DataFrame(X_train_PREPOST,
                               columns=PREPOST_feature_col_names)

In [145]:
X_train_PREPOST.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,-0.51204,0.60592,-0.75915,-0.74853,1.12084,-0.23094,0.15716,-0.08171,-0.29361,0.28612,-0.17594,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,1.65960,-2.48173,1.15944,-0.38721,0.0375,0.03141,0.61150,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,

### Transforming `X_dev_all` to get `X_dev_PREPOST`

In [146]:
# X_dev_PREPOST = PREPOST_feature_matrix.transform(X_dev_all)

- validating

In [147]:
# X_dev_PREPOST.shape, y_dev.shape

- putting into a `DataFrame` for easy analysis

In [148]:
# X_dev_PREPOST = pd.DataFrame(X_dev_PREPOST,
#                              columns=PREPOST_feature_col_names)

In [149]:
# X_dev_PREPOST.head()

### Transforming `X_test_all` to get `X_test_PREPOST`

In [150]:
X_test_PREPOST = PREPOST_feature_matrix.transform(X_test_all)

- validating

In [151]:
X_test_PREPOST.shape, y_test.shape

((850, 221), (850,))

- putting into a `DataFrame` for easy analysis

In [152]:
X_test_PREPOST = pd.DataFrame(X_test_PREPOST,
                              columns=PREPOST_feature_col_names)

In [153]:
X_test_PREPOST.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,0.57294,0.33838,-0.05818,-0.18295,0.92035,-0.11828,0.72170,0.12359,0.03349,0.55594,-0.33813,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,1.11008,0.91647,-0.77335,-0.22160,0.03750,0.03141,-0.53988,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0

## Modeling Scenarios Using `PREOP` and `POSTOP` Datasets

### - Scenario A: `Kitchen Sink` - Modeling the Combined `PREOP` and `POSTOP` Dataset

- code belows shows feature matrix assembly using `Pipeline` - could have used the already named `feature matricies` created above: `X_train_PREPOST`, `X_dev_PREPOST` and `X_test_PREPOST` in conjunction with their respective `y` vectors

#### `LogisticRegression()` with `GridSearchCV`

- instantiating `Pipeline`

In [154]:
PREPOST_modelA_pipe = Pipeline(steps=[
    ('get_PREPOST_feature_matrix', PREPOST_feature_matrix),
    ('Classifier', LogisticRegression())
    ])

- defining a parameter grid for `GridSearchCV`

In [155]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.001, 0.01, 1.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

- defining `scoring_metrics`

In [156]:
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

- create `GridSearchCV` object

In [157]:
gs = GridSearchCV(PREPOST_modelA_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

- fitting `GridSearchCV`

In [158]:
PREPOST_modelA = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:   57.9s remaining:   14.5s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.0min finished


- examining results

In [159]:
gs_results = pd.DataFrame(PREPOST_modelA.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)

In [160]:
display_cols = ['param_Classifier__C',
                'param_Classifier__penalty',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [161]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [162]:
gs_results_summary.rename(columns={'param_Classifier__C': 'C',
                                   'param_Classifier__penalty': 'penalty'},
                          inplace=True)

In [163]:
gs_results_summary

,C,penalty,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,0.01,l1,0.63875,0.72816,0.05123,0.02699,0.50797
1,1,l1,0.55265,0.90005,0.04629,0.02497,0.31741
2,0.001,l1,0.50000,0.50000,0.00000,0.00000,0.00000


#### Key Takeaways
- `LogisticRegression()` with `C=0.01` beats `STS`

## Scenario Ab - Just Preop Data Logistic

### Step 3b: Model and Evaluate Stacked Model Using `RandomForestClassifier`

In [164]:
PRE_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                       ('PREOP_cat_features', PREOP_categorical_features_pipeline)
                                      ])

In [165]:
PRE_modelA_pipe = Pipeline(steps=[
    ('get_PRE_feature_matrix', PRE_feature_matrix),
    ('Classifier', LogisticRegression())
    ])

In [166]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.0275, 0.01, 1.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

In [167]:
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

In [168]:
gs = GridSearchCV(PRE_modelA_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

In [169]:
PRE_modelA = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:   10.9s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   14.6s finished


In [170]:
gs_results = pd.DataFrame(PRE_modelA.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)
display_cols = ['param_Classifier__C',
                'param_Classifier__penalty',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)
gs_results_summary.rename(columns={'param_Classifier__C': 'C',
                                   'param_Classifier__penalty': 'penalty'},
                          inplace=True)

In [171]:
gs_results_summary

,C,penalty,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,0.0275,l1,0.61195,0.75081,0.04436,0.02335,0.44564
1,0.01,l1,0.60580,0.68259,0.04372,0.02285,0.51456
2,1,l1,0.53869,0.82717,0.03579,0.01899,0.31000


### - Scenario B: `Model Stacking` - Incorporate Predictions from `PREOP` Training with `POSTOP` Data
- this time, we will make use of pre-saved `X_train_PREOP` and `X_train_POSTOP` feature matricies created above via `Pipeline` 

In [172]:
X_train_PREOP.shape, X_train_POSTOP.shape, y_train.shape

((7650, 110), (7650, 111), (7650,))

#### Step 1.  Generate vector of probabilities of stroke for using `PREOP` features and cross validated training set
- `LogisticRegression` hyperparameters were tuned in a separate notebook

In [173]:
lr_predict_probs = cross_val_predict(LogisticRegression(penalty='l1',
                                                        C=0.0275,
                                                        class_weight='balanced',
                                                        random_state=0,
                                                        solver='liblinear'),
                                     X_train_PREOP,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished


- `cross_val_predict` returns an `numpy.ndarray` with `shape` `(n, 2)` as it returns a probability per class `(0, 1)`
- since we are interested in the `1` class, need to carve our the second column of the `numpy.ndarray`
- add `[:,1]`

- examining `lr_predict_probs`

In [174]:
type(lr_predict_probs)

numpy.ndarray

In [175]:
lr_predict_probs.shape

(7650, 2)

In [176]:
lr_predict_probs[0:5]

array([[0.6399836 , 0.3600164 ],
       [0.82819435, 0.17180565],
       [0.60969742, 0.39030258],
       [0.58204676, 0.41795324],
       [0.35985242, 0.64014758]])

In [177]:
lr_predict_probs[0:5, 1]

array([0.3600164 , 0.17180565, 0.39030258, 0.41795324, 0.64014758])

### Step 1a.  Generate additional vector of probabilities of stroke for using `PREOP` features and CV training set
- Not limited to stacking one `PREOP` model
- Running `RandomForestClassifier` `PREOP` model 
- hyperparameters were tuned in a separate notebook - using AP's highest scoring model
- going to assemble `X_train_PREOP_TREE` using `Pipelines`
- can utilize existing `PREOP_numerical_features_pipeline`
- need to define `PREOP_categorial_TREE_features_pipeline`
- use`FeatureUnion` to combine to create `PREOP` feature matrix usable for `RandomForestClassifier`

- creating `Pipeline` to assemble `PREOP_categorical_TREE_features_pipeline`

In [178]:
PREOP_categorical_TREE_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_categorical_TREE_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

In [179]:
PREOP_feature_matrix_TREE = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                          ('PREOP_cat_tree_features', PREOP_categorical_TREE_features_pipeline)
                                         ])

- combining with `PREOP_numerical_features_pipeline` defined above to assemble `PREOP_feature_matrix_TREE`
- if plan is to assemble `feature_matrix` but not use in a modeling pipeline as is done below, then can assign to a variable name like `X_train_PREOP_TREE` -- did this above for `X_train_PREOP` 

#### Modeling `PREOP_dataset_TREE` using `RandomForestClassifier()`

- creating `PREOP_RF_CLF_pipe`

In [180]:
PREOP_RF_clf_pipe = Pipeline(steps=[
    ('get_PREOP_TREE_feature_matrix', PREOP_feature_matrix_TREE),
    ('Classifier', RandomForestClassifier(n_estimators=315,
                                          max_features=0.1,
                                          max_depth=None,
                                          min_samples_split=50,
                                          class_weight='balanced',
                                          random_state=0))])

- in this scenario - assuming already have tuned the hyperparamenters
- alternatively, could have run a `GridSearchCV` first to tune the hyperparameters and then use the `best_estimator_` to get estimated probabilities of stroke with the code below

- running `RandomForestClassifer`

In [181]:
rf_predict_probs = cross_val_predict(PREOP_RF_clf_pipe,
                                     X_train_all,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   10.7s remaining:   16.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   18.4s finished


- examining `rf_predict_probs`

In [182]:
rf_predict_probs.shape

(7650, 2)

In [183]:
rf_predict_probs[0:5, 1]

array([0.02858866, 0.06246313, 0.12276119, 0.11960992, 0.19616412])

### Step 2: Incorporate vector of predicted probabilities into `X_train_POSTOP`
- adding `lr_predict_probs`
- adding `rf_predict_probs`
- adding `STS` model predicted probability using `predstro_train`

In [184]:
X_train_POSTOP_STACKED = X_train_POSTOP.copy()

In [185]:
X_train_POSTOP_STACKED.shape, X_train_POSTOP.shape

((7650, 111), (7650, 111))

- adding vectors of `predict_probas` from `PREOP` models and `STS` models to `POSTOP` feature matrix

In [186]:
X_train_POSTOP_STACKED['LR_PREOP_PROBA'] = lr_predict_probs[:, 1]
X_train_POSTOP_STACKED['RF_PREOP_PROBA'] = rf_predict_probs[:, 1]
#X_train_POSTOP_STACKED['STS_predstro'] = predstro_train

In [187]:
X_train_POSTOP_STACKED.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,LR_PREOP_PROBA,RF_PREOP_PROBA
0,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,1.65960,-2.48173,1.15944,-0.38721,0.0375,0.03141,0.61150,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36002,0.02859
1,0.0,-0.09801,-0.09217,-0.00525,-0.10154,6.73500,2.18333,-0.95864,-0.03123,-0.51176,3.86140,-0.30441,0.0375,0.03141,0.92041,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.17181,0.06246
2,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,-0.66530,0.02998,0.52833,-0.55283,0.0375,0.03141,0.47109,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.39030,0.12276
3,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,-0.49621,-0.56101,0.46916,0.52367,0.0375,0.03141,0.13410,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.41795,0.11961
4,0.0,-0.09801,-0.09217,-0.00525,-0.10154,2.09268,-0.26517,-1.96242,-0.03123,0.47322,0.86361,0.02682,0.0375,0.03141,1.45398,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0

In [188]:
X_train_POSTOP_STACKED.shape

(7650, 113)

### Step 3a: Model and Evaluate `X_train_POSTOP_STACKED` Using `LogisticRegression()`

- instantiating a `LogisticRegression()` object

In [189]:
log_reg_clf = LogisticRegression()

- will use scoring metrics defined above in previous `GridSearchCV`
- defining hyperparameter grid

In [190]:
hyperparameters = {'penalty': ['l1'],
                   'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
                   'class_weight': ['balanced'],
                   'solver': ['liblinear'],
                   'random_state': [0]
                  }

- create `GridSearchCV` object

In [191]:
gs_stacked = GridSearchCV(log_reg_clf, #classifier
                          param_grid=hyperparameters, #param_grid
                          scoring=scoring_metrics, #loss functions
                          cv=5, #number of folds, default=`StratifiedKFold`
                          refit='roc_auc', # best_estimator_ will be based on this scoring metric
                          return_train_score=True,
                          n_jobs=-1,
                          verbose=5)

- fitting `GridSearchCV`

In [192]:
PREPOST_modelB = gs_stacked.fit(X_train_POSTOP_STACKED, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.9min finished


- examining results

In [193]:
gs_results = pd.DataFrame(PREPOST_modelB.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                      ascending=False)

In [194]:
display_cols = ['param_C',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [195]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [196]:
gs_results_summary

,param_C,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,0.01,0.66110,0.70125,0.05726,0.03021,0.55272
1,0.1,0.61061,0.75927,0.05595,0.02976,0.46979
2,1,0.58609,0.78507,0.04849,0.02586,0.39346
3,10,0.57692,0.78991,0.05057,0.02701,0.40060
4,0.0001,0.50000,0.50000,0.00000,0.00000,0.00000
5,0.001,0.50000,0.50000,0.00000,0.00000,0.00000


### Key Takeaways
- similar to `Modeling Scenario A` achieved better results than `STS` with the `POSTOP` features
- results were slightly below that of `Modeling Scenario A` on a `mean_test_roc_auc` basis
- extend this stacking concept to add additional vectors of probability of stroke from other models `RandomForest` and `STS`
- adding the `RandomForest` predicted probabilities improved results slightly, but the addition of `STS` on top of that did not improve the `mean_test_roc_auc`
- instead of modeling the stacked `POSTOP` data with `LogisticRegression` can try another model such as `RandomForest` or `SVM`
- in addition can also try using `PolynomialFeatures` transformation using `Scenario B` to see if any benefit from modeling the interaction of the `PREOP` predictions with the additional `POSTOP` features
- should examine the non-zero coeficients in `POSTOP` to get insight into what `POSTOP` features are useful in enhancing stroke prediction

## Step 3 ab - Just Preop Data RF

In [197]:
PREOP_RF_clf_pipe = Pipeline(steps=[
    ('get_PREOP_TREE_feature_matrix', PREOP_feature_matrix_TREE),
    ('Classifier', RandomForestClassifier())])

In [198]:
parameter_grid = {'Classifier__n_estimators': [315],
                  'Classifier__max_features': [0.1],
                  'Classifier__max_depth': [None],
                  'Classifier__min_samples_split': [50],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__random_state': [0]
                 }
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

In [199]:
gs = GridSearchCV(PREOP_RF_clf_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

In [200]:
PRE_modelRF = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   23.2s remaining:   34.8s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   38.6s finished


In [201]:
gs_results = pd.DataFrame(PRE_modelRF.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)
display_cols = ['param_Classifier__n_estimators',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)
gs_results_summary.rename(columns={'param_Classifier__n_estimators': 'n_estimators'},
                          inplace=True)

In [202]:
gs_results_summary

,n_estimators,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,315,0.61038,1.0,0.0,0.0,0.0


### Step 3b: Model and Evaluate Stacked Model Using `RandomForestClassifier`

#### Creating `POSTOP_feature_matrix_TREE` using `sklearn` `Pipelines` 

- can use similar coding pattern to assemble `POSTOP_feature_matrix_TREE` if needed
    - `POSTOP_numerical_features_pipeline` has been previously created above
    - would need to create `POSTOP_categorical_TREE_features_pipeline` -- THIS HAS NOT BEEN DONE YET
    - then use `FeatureUnion()` to assemble `POSTOP_feature_matrix_TREE`
    - similar to what was done above in using `Pipeline`s to create `X_train_PREOP`, `X_train_POSTOP` and corresponding `X_dev` and `X_test` 

In [203]:
POSTOP_categorical_TREE_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_categorical_TREE_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- using `FeatureUnion` combining using with already created `POSTOP_numerical_features_pipeline` to assemble `POSTOP_feature_matrix_TREE`

In [204]:
POSTOP_feature_matrix_TREE = FeatureUnion([('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                           ('POSTOP_cat_tree_features', POSTOP_categorical_TREE_features_pipeline)
                                          ])

#### NOTE - as we did above:

- `fit_transform` only on `X_train_all` for `X_train_POSTOP_TREE`
- `transform` on `X_dev_all` and `X_test_all` for `X_dev_POSTOP_TREE` and `X_test_POSTOP_TREE`
- assign compreleted `feature_matrix` to variable name if not using directly in a `Pipeline` with classifier step

#### Transforming `X_train_all` to get `X_train_POSTOP_TREE`

In [205]:
X_train_POSTOP_TREE = POSTOP_feature_matrix_TREE.fit_transform(X_train_all)

- validating

In [206]:
X_train_all.shape, X_train_POSTOP_TREE.shape, y_train.shape

((7650, 247), (7650, 62), (7650,))

- putting into a `DataFrame` for easy analysis

In [207]:
POSTOP_feature_TREE_col_names = POSTOP_numerical_col_names + POSTOP_categorical_TREE_col_names

In [208]:
len(POSTOP_feature_TREE_col_names), len(POSTOP_numerical_col_names), len(POSTOP_categorical_TREE_col_names)

(62, 15, 47)

In [209]:
X_train_POSTOP_TREE = pd.DataFrame(X_train_POSTOP_TREE,
                                   columns=POSTOP_feature_TREE_col_names)

In [210]:
X_train_POSTOP_TREE.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,1.65960,-2.48173,1.15944,-0.38721,0.0375,0.03141,0.61150,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,-0.09801,-0.09217,-0.00525,-0.10154,6.73500,2.18333,-0.95864,-0.03123,-0.51176,3.86140,-0.30441,0.0375,0.03141,0.92041,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,-0.66530,0.02998,0.52833,-0.55283,0.0375,0.03141,0.47109,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,-0.49621,-0.56101,0.46916,0.52367,0.0375,0.03141,0.13410,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,-0.09801,-0.09217,-0.00525,-0.10154,2.09268,-0.26517,-1.96242,-0.03123,0.47322,0.86361,0.02682,0.0375,0.03141,1.45398,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Creating `X_train_POSTOP_TREE_STACKED`
- adding vectors of `predict_probas` from `PREOP` models and `STS` models to `POSTOP_TREE` feature matrix

In [211]:
X_train_POSTOP_TREE_STACKED = X_train_POSTOP_TREE.copy()

In [212]:
X_train_POSTOP_TREE_STACKED.shape, X_train_POSTOP_TREE.shape

((7650, 62), (7650, 62))

In [213]:
X_train_POSTOP_TREE_STACKED['LR_PREOP_PROBA'] = lr_predict_probs[:, 1]
X_train_POSTOP_TREE_STACKED['RF_PREOP_PROBA'] = rf_predict_probs[:, 1]
#X_train_POSTOP_TREE_STACKED['STS_predstro'] = predstro_train

In [214]:
X_train_POSTOP_TREE_STACKED.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn,LR_PREOP_PROBA,RF_PREOP_PROBA
0,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,1.65960,-2.48173,1.15944,-0.38721,0.0375,0.03141,0.61150,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36002,0.02859
1,0.0,-0.09801,-0.09217,-0.00525,-0.10154,6.73500,2.18333,-0.95864,-0.03123,-0.51176,3.86140,-0.30441,0.0375,0.03141,0.92041,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.17181,0.06246
2,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,-0.66530,0.02998,0.52833,-0.55283,0.0375,0.03141,0.47109,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.39030,0.12276
3,0.0,-0.09801,-0.09217,-0.00525,-0.10154,-0.22847,-0.26517,0.04514,-0.49621,-0.56101,0.46916,0.52367,0.0375,0.03141,0.13410,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.41795,0.11961
4,0.0,-0.09801,-0.09217,-0.00525,-0.10154,2.09268,-0.26517,-1.96242,-0.03123,0.47322,0.86361,0.02682,0.0375,0.03141,1.45398,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.64015,0.19616


In [215]:
X_train_POSTOP_TREE_STACKED.shape

(7650, 64)

- instantiating a `RandomForestClassifier()` object
- code pattern from line 169 - `GridSearchCV`

In [216]:
rf_clf = RandomForestClassifier(class_weight='balanced',
                                random_state=0)

In [217]:
hyper_params = {'n_estimators': [100, 200, 250, 300],          # default is `10`
                'max_features': ['auto', 0.10, 0.25, 0.50],    # default is square root of `n_features`    
                'max_depth': [2, 5, 10, None],                 # default is `None`
                'min_samples_split': [2, 5, 10]}               # default is `2`

In [218]:
gs = GridSearchCV(rf_clf, # estimator
                  hyper_params, # param_grid
                  scoring=scoring_metrics, # loss functions
                  cv=5, 
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  verbose=2,
                  return_train_score=True,
                  n_jobs=-1)

In [219]:
rf_gs_stacked_model = gs.fit(X_train_POSTOP_TREE_STACKED, 
                             y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 48.7min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 67.7min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 172.1min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 398.2min finished


- `GridSearchCV` takes approximately 90 minutes to run

#### Saving `rf_gs_stacked_model`

In [220]:
joblib.dump(rf_gs_stacked_model, 'rf_gs_stacked_model_valve_nosts_12_2.sav')

['rf_gs_stacked_model_valve_nosts_12_2.sav']

#### Loading saved `rf_gs_stacked_model`

In [221]:
#rf_gs_stacked_model = joblib.load('rf_gs_stacked_model_11_17.sav')

- analyzing results

In [222]:
rf_gs_stacked_model.best_params_

{'max_depth': 2,
 'max_features': 0.1,
 'min_samples_split': 10,
 'n_estimators': 250}

In [223]:
gs_results = pd.DataFrame(rf_gs_stacked_model.cv_results_)

In [224]:
gs_results = gs_results.sort_values(by=['mean_test_roc_auc'],
                                           ascending=False)

In [225]:
display_cols = ['params',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [226]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [227]:
gs_results_summary.head()

,params,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,"{'max_depth': 2, 'max_features': 0.1, 'min_samples_split': 10, 'n_estimators': 250}",0.69101,0.76855,0.06744,0.03617,0.49972
1,"{'max_depth': 2, 'max_features': 0.1, 'min_samples_split': 5, 'n_estimators': 250}",0.69029,0.76844,0.06731,0.03610,0.49972
2,"{'max_depth': 2, 'max_features': 0.1, 'min_samples_split': 10, 'n_estimators': 200}",0.69026,0.76874,0.06555,0.03517,0.48434
3,"{'max_depth': 2, 'max_features': 0.1, 'min_samples_split': 5, 'n_estimators': 200}",0.69004,0.76855,0.06546,0.03512,0.48434
4,"{'max_depth': 2, 'max_features': 0.1, 'min_samples_split': 2, 'n_estimators': 250}",0.69000,0.76855,0.06623,0.03551,0.49231


#### Key Takeaways
- better than `STS` alone, but results slightly below that of `LogisticRegression()`

## Results on test set

In [228]:
def calcTestROC(model, X_test, y_test):
    pred_prob = model.predict_proba(X_test)[:,1]
    return roc_auc_score(y_test, pred_prob)

In [229]:
calcTestROC(PREPOST_modelA, X_test_all, y_test ) #log reg on pre+post data

0.6891816367265469

In [230]:
calcTestROC(PRE_modelA, X_test_all, y_test) #log reg on pre data

0.5829940119760479

In [231]:
calcTestROC(PRE_modelRF, X_test_all, y_test ) #RF on pre data

0.5974451097804391

In [232]:
#need to create X_test_POSTOP_STACKED
X_test_POSTOP_STACKED = X_test_POSTOP.copy()
X_test_POSTOP_STACKED['LR_PREOP_PROBA'] = PRE_modelA.predict_proba(X_test_all)[:, 1]
X_test_POSTOP_STACKED['RF_PREOP_PROBA'] = PRE_modelRF.predict_proba(X_test_all)[:, 1]

In [233]:
calcTestROC(PREPOST_modelB, X_test_POSTOP_STACKED, y_test ) #log reg on pre+post stacked

0.6906187624750499

In [234]:
#need to create X_test_POSTOP_TREE_STACKED
X_test_POSTOP_TREE = POSTOP_feature_matrix_TREE.transform(X_test_all)
X_test_POSTOP_TREE = pd.DataFrame(X_test_POSTOP_TREE,
                                   columns=POSTOP_feature_TREE_col_names)
X_test_POSTOP_TREE_STACKED = X_test_POSTOP_TREE.copy()
X_test_POSTOP_TREE_STACKED['LR_PREOP_PROBA'] = PRE_modelA.predict_proba(X_test_all)[:, 1]
X_test_POSTOP_TREE_STACKED['RF_PREOP_PROBA'] = PRE_modelRF.predict_proba(X_test_all)[:, 1]

In [235]:
calcTestROC(rf_gs_stacked_model, X_test_POSTOP_TREE_STACKED, y_test ) #RF on pre+post stacked

0.6665069860279441

In [237]:
round(roc_auc_score(y_test,predstro_test),5)

0.74116

In [236]:
break

SyntaxError: 'break' outside loop (<ipython-input-236-6aaf1f276005>, line 4)

### `Polynomial` Transformation on Stacked Model
- to see if adding interaction terms between `PREOP` models predicted probabilities and `POSTOP` features can improve results

- instantiating `Pipeline` that will include `Polynomial` transformation of `X_train_POSTOP_STACKED`

In [ ]:
poly_lr_model_pipe = Pipeline(steps=[
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('Classifier', LogisticRegression())
    ])

- defining a parameter grid for `GridSearchCV`

In [ ]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.0001, 0.001, 0.01, 0.10, 0.50, 1.0, 10.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

- create `GridSearchCV` object

In [ ]:
gs = GridSearchCV(poly_lr_model_pipe, # estimator
                  param_grid=parameter_grid, # param_grid
                  scoring=scoring_metrics, # loss functions
                  cv=5,
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  verbose=2,
                  return_train_score=True,
                  n_jobs=-1)

- fitting `GridSearchcV`

- `GridSearchCV` takes over 2 full hours to run

#### Saving `poly_log_reg_lasso`

In [ ]:
#joblib.dump(poly_log_reg_lasso, 'poly_log_reg_lasso_model_11_17.sav')

#### Loading `poly_log_reg_lasso`

In [ ]:
poly_log_reg_lasso = joblib.load('poly_log_reg_lasso_model_11_17.sav')

- examining results

In [ ]:
poly_log_reg_lasso.best_params_

In [ ]:
gs_results = pd.DataFrame(poly_log_reg_lasso.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                      ascending=False)

In [ ]:
display_cols = ['params',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [ ]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [ ]:
gs_results_summary

#### Key Takeways
- `PolynomialTransformation` actually had worse results than non-transformed `LogisticRegression`
- Results were in-line with `STS` model

## CODE BELOW IS USEFUL FOR DIAGNOSTIC CHARTS ON MODEL SCORING

### Need to modify and adapt for code above given changes in variable names

In [ ]:
break

# ---------------------------------------------------------------------------------------------

## Visualizing the Effect of `LogisticRegression()` Hyperparameter Values
- See Albon `11.13` pages 205-207
- range of values for `C` - inverse of regularization strength

In [ ]:
param_values = np.linspace(0.001, 0.05, 25)

- calculating `roc_auc` on training and test set using `param_range` for `C`

In [ ]:
train_scores, test_scores = validation_curve(LogisticRegression(penalty='l1',
                                                                class_weight='balanced',
                                                                random_state=0,
                                                                solver='liblinear'), #classifier
                                             X_train, #feature matrix
                                             y_train, #target vector
                                             param_name='C', #hyperparameter to examine,
                                             param_range=param_values, #range of hyperparameter values
                                             cv=5, #number of folds, default=`StratifiedKFold`
                                             scoring='roc_auc', #scoring metric
                                             n_jobs=-1, #use all computer cores
                                             verbose=5) 

- calculate `mean` and `standard deviation` for `training` set scores

In [ ]:
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

- calculate `mean` and `standard deviation` for `test` set scores

In [ ]:
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

- Plotting

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')

# plot mean accuracy scores for training and test sets
plt.plot(param_values,
         train_mean,
         label='Training Score',
         color='midnightblue')

plt.plot(param_values,
         test_mean,
         label='Cross Validation Score',
         color='darkviolet')

# plot accuracy bands for training and test sets
plt.fill_between(param_values, 
                 train_mean - train_std,
                 train_mean + train_std,
                 color='lightsteelblue')

plt.fill_between(param_values,
                 test_mean - test_std,
                 test_mean + test_std,
                 color='thistle')

# create plot
plt.title('Validation Curve for Logistic Regression with l1 Regularization')
plt.xlabel('C')
plt.ylabel('AUCROC')
plt.legend(loc='lower right')
plt.show()       

In [ ]:
validation_summary = pd.DataFrame({'C': param_values,
                                   'mean_test_roc_auc': test_mean,
                                   'mean_train_roc_auc': train_mean})

In [ ]:
validation_summary[validation_summary['mean_test_roc_auc'] == np.max(validation_summary['mean_test_roc_auc'])]

#### How does this tuned model compare to the baseline `STS` Model

In [ ]:
round(roc_auc_score(y_train, predstro_train), 5)

#### Key Takeaways
- Fine tuning of hyperparmeter `C` results in `C=0.0275`
- Results in-line (but still lower) than `STS` Model

## Precision-Recall Curve for Tuned Logistic Regression Model
- `Hands On Machine Learning with Scikit-Learn & Tensorflow` by Aurelien Geron pages 83-93
#### How `cross_val_predict` works:
- `cross_val_predict` returns for each element in the input, the prediction that was obtained for that element when it was in the `test` set

- generating vector of probabilities of stroke for cross validated training set

In [ ]:
lr_predict_probs = cross_val_predict(LogisticRegression(penalty='l1',
                                                        C=0.0275,
                                                        class_weight='balanced',
                                                        random_state=0,
                                                        solver='liblinear'),
                                     X_train,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

- using `lr_predict_probs` can compute precision and recall for all possible thresholds using the `precision_recall_curve` function
- returns an `numpy.ndarray` with `shape` `(n, 2)` as it returns a probability per class `(0, 1)`
- since we are interested in the `1` class, need to carve our the second column of the `numpy.ndarray`
- add `[:,1]`

In [ ]:
type(lr_predict_probs)

In [ ]:
lr_predict_probs.shape

In [ ]:
lr_predict_probs[0:5]

In [ ]:
lr_predict_probs[0:5, 1]

#### Generating `precision_recall_curve`

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y_train,
                                                         lr_predict_probs[:, 1])

- `precision` values such that element `i` is the the `precision` of predictions with `score>=threholds[i]` and the last element is `1`

In [ ]:
precisions[0:5] # first 5 elements of the numpy array

In [ ]:
precisions[-5:] # last 5 elements of the numpy array

- decreasing `recall` values such that element `i` is the the `precision` of predictions with `score>=threholds[i]` and the last element is `0`

In [ ]:
recalls[0:5]

In [ ]:
recalls[-5:]

- increasing `thresholds` on the decision function used to compute `precision` and `recall`

In [ ]:
thresholds[0:5]

In [ ]:
thresholds[-5:]

### Plotting `precision` and `recall` as functions of `threshold`

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], 'b--', label='Precision')
    plt.plot(thresholds, recalls[:-1], 'g-', label='Recall')
    plt.title('Precision and Recall versus Threshold')
    plt.xlabel('Threshold')
    plt.legend(loc='upper right')
    plt.ylim([0, 1])

### Precision and Recall versus Threshold for Logistic Regression Model

In [ ]:
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

- helper function

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx], idx

- examining specific `threshold`, `Recall` and `Precision` trade-offs
- suppose you wanted a `recall` of `0.80`, what `threshold` will you need and what will the `precision` be?

In [ ]:
find_nearest(recalls, 0.80)

In [ ]:
print('{0:15} {1:5f}'.format('Recall:', recalls[14806]))
print('{0:15} {1:5f}'.format('Precision:', precisions[14806]))
print('{0:15} {1:5f}'.format('Threshold:', thresholds[14806]))

- see `Geron` page 90 for additional details

### Precision and Recall versus Threshold for STS Model

In [ ]:
sts_precisions, sts_recalls, sts_thresholds = precision_recall_curve(y_train,
                                                                     predstro_train)

In [ ]:
plot_precision_recall_vs_threshold(sts_precisions, sts_recalls, sts_thresholds)
plt.show()

## Precision-Recall Curves
- `Introduction to Machine Learning with Python` by Andreas C. Muller and Sarah Guido pages 289-296
- the closer a `precision-recall curve` stays to the upper-right corner, the better classifier

In [ ]:
def plot_precision_recall_curve(precisions, recalls):
    plt.plot(precisions,
             recalls,
             color='blue',
             label='Precision-Recall Curve')
    plt.title('Precision-Recall Curve')
    plt.ylabel('Recall')
    plt.xlabel('Precision')
    plt.legend(loc='upper right')

### `Precision-Recall Curve` for `LogisticRegression` Model

In [ ]:
plot_precision_recall_curve(precisions, recalls)
plt.show()

#### Area Under the `Precision-Recall Curve` for `LogisticRegression` Model

In [ ]:
round(auc(recalls, precisions), 5)

### `Precision-Recall Curve` for `STS` Model

In [ ]:
plot_precision_recall_curve(sts_precisions, sts_recalls)
plt.show()

#### Area Under the `Precision-Recall Curve` for `STS` Model

In [ ]:
round(auc(sts_recalls, sts_precisions), 5)

#### Key Takeaways
- `STS` model is superior to `LogisticRegression` model looking at `Precision-Recall Curves` and `PR AUC`

### Comparing `Logistic Regression` and `STS` Model `Precision-Recall` Curves

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')

# plotting `LogisticRegression` Precision-Recall Curve
plt.plot(precisions,
         recalls,
         'b--',
         label='Logistic Regression Model')

# plotting `STS Model` Precision-Recall Curve
plt.plot(sts_precisions,
         sts_recalls,
         'g-',
         label='STS Model')

plt.title('Logistic Regression, STS Model Precision-Recall Curves')
plt.ylabel('Recall')
plt.xlabel('Precision')
plt.legend(loc='upper right')
plt.xlim([0, 0.10]) # cut off values > 0.10 because they were essentially zero
plt.show()

#### Key Takeaways
- plot clearly shows `STS` model outperperformance over `LogisticRegression`
- can use basic code here to plot `Precision-Recall Curves` from other models on the same plot for comparison

## Receiver Operating Characteristic (`ROC`) Curves
- Albon `11.5` pages 189-192
- a classifier that predicts every observation correctly would look like the solid `gray` line in the plotted `ROC` curves below
- the solid `gray` line goes straight up to the top immediately
- a classifier that predicts at random will appear as the diagonal line
- the better the model, the closer it is to the solid `gray` line
- the `ROC` curve represents the respective `TPR` and `FPR` for every probability threshold

- creating `true` and `false` probabilities

In [ ]:
false_positive_rate, true_positive_rate, threshold = roc_curve(y_train,
                                                               lr_predict_probs[:, 1])

- `plot_roc_curve` function

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.title('Receiver Operating Curve (ROC)')
    plt.plot(fpr, tpr)
    plt.plot([0, 1], ls='--')
    plt.plot([0, 0],
             [1, 0],
             c='0.7'),
    plt.plot([1, 1], c='0.7')
    plt.ylabel('True Positive Rate (Recall)')
    plt.xlabel('False Positive Rate')

#### `LogisiticRegression` Model `ROC` Curve

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')
plot_roc_curve(false_positive_rate, true_positive_rate)
plt.show()

In [ ]:
round(roc_auc_score(y_train, lr_predict_probs[:, 1]), 5)

- suppose you want `recall` in a certain range - what is the `false positive rate` you would have to accept?

In [ ]:
lr_roc_curve = pd.DataFrame({'Recall': true_positive_rate,
                             'FPR': false_positive_rate})

- assume you want `recall` around `0.80`

In [ ]:
lr_roc_curve[lr_roc_curve['Recall'].between(0.799, 0.801)]

- could have also used the `find_nearest` helper function above

In [ ]:
lr_roc_curve[lr_roc_curve['Recall'] == (find_nearest(lr_roc_curve['Recall'], 0.80)[0])]

- suppose you want to know `recall` and `FPR` for a given threshold

In [ ]:
np.where(threshold == 0.10)[0]

In [ ]:
print('{0:15} {1:5f}'.format('Threshold:', threshold[11]))
print('{0:15} {1:5f}'.format('Recall:', true_positive_rate[11]))
print('{0:15} {1:5f}'.format('FPR:', false_positive_rate[11]))

#### `STS` Model `ROC` Curve

In [ ]:
sts_fpr, sts_tpr, threshold = roc_curve(y_train,
                                        predstro_train)

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')
plot_roc_curve(sts_fpr, sts_tpr)
plt.show()

In [ ]:
round(roc_auc_score(y_train, predstro_train), 5)

#### Key Takeaways
- `STS` Model slightly outperforms `LogisticRegression`